In [2]:
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re 
from tqdm import tqdm_notebook as tqdm

"""
集めるデータごとに関数を作成する。→ gather_information()で全部集める。
URLにアクセスする系は関数だとその度アクセスしてしまうので関数の中に組み込むのはよくない。
"""

In [3]:
#J1各チームの日程・結果にアクセスするためのurlリスト。
html_all = urlopen("https://soccer.yahoo.co.jp/jleague/teams/j1")
soup_all = BeautifulSoup(html_all, "html.parser")
team_url = soup_all.find_all(href = re.compile("^/jleague/teams/schedule"))
team_url_list= []
for a in team_url:
    team_url_list.append("https://soccer.yahoo.co.jp" + a.get("href"))

In [4]:
#J1チームリストの作成
team_name_list = []
for i in tqdm(team_url_list):
    html = urlopen (i)
    soup = BeautifulSoup(html, "html.parser")
    team_name = soup.find_all(class_ = "name")
    for i in team_name:
        team_name = i.text
        team_name_list.append(team_name)

In [5]:
def score(soup):
    #各試合のスコアのTag
    score_tag = soup.find_all(href =  re.compile("^/jleague/game/score/"))
    #Tagからスコアをget
    score_list = []
    for v in score_tag:
        score_list.append(v.text)
    #文字化け？を修正するため一旦文字列にする。
    score_list_str = ",".join(score_list).replace("\xa0","")
    score_list = score_list_str.split(",")
    return(score_tag, score_list)

In [6]:
#結果を格納しているタグを発見してテキストにする。
def results(soup):
    results = soup.find_all(class_="textC")
    #勝敗だけ抽出。
    results_list = []
    for v in results:
        if (v.text == "△" or v.text == "●" or  v.text == "○" ):
            results_list.append(v.text)
    #リストの勝敗を数値に変換。
    results_list = ["2" if "△" in s  else s for s in results_list]
    results_list = ["1" if "○" in s  else s for s in results_list]
    results_list = ["0" if "●" in s  else s for s in results_list]
    return(results_list)

In [7]:
def home_review(score_tag):
    #Home/Away
    home_away_url = []
    home_away_list = []
    #review
    match_review = []
    #試合詳細のURLをリスト化
    for a in score_tag:
        home_away_url.append("https://soccer.yahoo.co.jp"  + a.get("href"))
    #リスト化されたURLからHOME/AWAY情報を抽出
    for i in tqdm(home_away_url):
        home_away_html = urlopen(i)
        home_away_soup = BeautifulSoup(home_away_html, "html.parser")
        if str(home_away_soup.title.contents[0]).split()[-7] == team_name :
            home_away_list.append(1)
        else:
            home_away_list.append(0)
        review_test = home_away_soup.find("p",class_="", text=re.compile("。")).text
        pattern = re.compile("掲載")
        matchObj = pattern.search(review_test)
    #掲載　という文字が入っていたらレビューではないので空にする。
        if matchObj:
            match_review.append("Nah")
        else:
            match_review.append(review_test)
    return(home_away_url, home_away_list, match_review)

In [121]:
###xに入るのは数字
###team x の情報を集める関数。
def gather_information(x):
    #各チームにアクセス。
    html = urlopen(team_url_list[x])
    soup = BeautifulSoup(html, "html.parser")
    #チーム名
    team_name = team_name_list[x]    
    df_score = score(soup)
    score_tag = df_score[0]
    df_results = results(soup)
    df_home_review = home_review(score_tag)
    #DataFrame→CSV にして出力
    df = pd.DataFrame({"results" : df_results,
                                     "home" : df_home_review[1],
                                     "score" : df_score[1],
                                       "match review" : df_home_review[2]
                                  })
    df.to_csv("/Users/yamadaikuya/pythonista/toto/results/"+team_name+".csv",sep=",")

In [144]:
gather_information(0)

In [123]:
show_dataframe(0)

,results,home,score,match review
0,0,0,1-0,復活を目指す広島と、さらなるステップアップを見据える札幌。ともに新しい指揮官を迎えて臨んだ一戦は、充実の内容を見せた。広島は柏と川辺の両サイドを中心に主導権を握ると、前半28分にJリーグデビュー戦のティーラシンが先制点を奪う。その後は守勢に回る時間が続いたが、ボールホルダーへのプレスを徹底。90分間走り抜いて勝点3を手に入れた。一方、良い攻めを見せたものの最後の精度が足りず1点に泣いた札幌。だが、ピッチを広く使って攻め立てる「ミシャサッカー」の浸透ぶりを随所に見せ、今後への期待を持たせた。
1,2,0,3-3,ホームのC大阪は前半からプレスの強度を高く保ち、奪えばすぐにカウンターへ持ち込む。パス主体の攻撃でチャンスをうかがう札幌のミスを誘ってゲームの主導権を握ると、今季から加入した高木の得点を含む2点を奪い、試合を折り返す。だが、リードを得たことでディフェンスに対する意識が強くなり、最終ラインをずるずると下げてしまう。時間を追うごとにゴール前のマークが甘くなっていくと、後半だけでアウェイチームに3ゴールを献上。勝点1を分け合ったものの、守備面のもろさを露呈して今後の連戦に不安が残る結果となった。
2,0,0,3-0,Nah
3,0,0,1-3,ポゼッション対ショートカウンターと、互いのスタイルが明確に出た一戦となった。今季初のホームゲームに臨んだ札幌は、清水のコンパクトなブロックを逆手に取るように、ピッチを広く使った攻撃を見せる。フリーで受けて正確なクロスを供給する三好、相手のギャップに顔を出し続けるチャナティップの2人が、両ウイングバックと連動して守備網を切り裂いた。しかし、清水の素早い切り替えに守備陣が対応し切れず、時間とともに失点を重ねることに。チームの形はともに表現できただけに、その完成度の差が結果として如実に表れてしまった。
4,0,0,0-1,Nah
5,1,0,2-1,今季初勝利を狙う札幌。パスをつなぎながら両サイドからのクロスとジェイの高さを起点にして攻撃を仕掛けるが、得点を奪えぬまま試合を折り返す。後半に入ると、チャナティップが躍動。後半8分に体の強さを生かしてDFをかわし、ジェイに縦パスを送ると、ジェイがこれを決めて先制する。同点で迎えた同49分には、ジェイが頭で折り返したボールに反応して劇的なヘディングシュートを決め、今季初の勝点3に大きく貢献した。対する長崎は、一時は同点に追い付いたが、逆転ゴールを奪えず。次節でJ1初勝利を挙げたい。
6,2,0,0-0,前半の鹿島は辛抱の時間が続く。開始直後の都倉のシュートを皮切りに、札幌の連動したスプリントで自陣深くにボールを運ばれる。特に駒井にサイドを破られて肝を冷やす場面が続出し、ディフェンスラインの集中力が問われる展開となる。見せ場は作るが、それ以上のピンチを迎え、後半の選手交代でも流れは変わらず。スコアレスドローの決着となった。対して札幌は、リスクを背負った攻撃で優位に進め、カシマでのリーグ戦初勝利は目前に迫ったが、相手の守備陣の粘りに遭って得点には至らず。鬼門での初の勝点は1となった。
7,0,0,2-1,Nah
8,1,0,3-0,カウンターを狙う札幌とボールを回す名古屋という互いの狙いが前半から交錯する試合展開。ここで持ち味を発揮したのはホームチームだった。右サイドで駒井と三好を中心にカウンターからチャンスを作ると、相手はCKに逃れることに精いっぱい。前半26分にはそのCKから先制点を奪うと、ここから一気に主導権は札幌へ。後半に入っても勢いはとどまらず、エース都倉がバイシクルで豪快に追加点をたたき込んだ。最後は古巣対決となった櫛引のオウンゴールもあり、3点を奪っての完勝。リーグ戦2試合ぶりの勝点3を手にした。
9,1,0,1-0,平日にも関わらず、10000人を超える観衆が駆け付けた一戦。札幌は、前半から都倉や三好を中心に相手ゴールを脅かした。だが、決定的なチャンスを生かせず、0－0のまま試合を折り返す。後半に入っても、勢いそのままに攻め続けるが、ネットを揺らせず、アディショナルタイムに突入する。このまま終了かと思われたが、後半46分に都倉が均衡を打破する貴重な1点を奪取。札幌ドには多くのサポーターの歓声が響き渡り、選手にとっても観客にとってもうれしい勝点3となった。


In [146]:
#全てのチームに関して情報を集める。
def gather_all_information():
    for i in tqdm(range(len(team_url_list))):
        gather_information(i)

In [147]:
gather_all_information()

In [140]:
#csvからdfに戻す。
def show_dataframe(i):
    df_return = pd.read_csv("/Users/yamadaikuya/pythonista/toto/results/"+team_name_list[i]+".csv", 
                      index_col=[0])
    df_return_latest = df_return[-1:]
    pd.options.display.max_colwidth = 1000 #最大文字数の変更
    pd.options.display.max_rows = 1000 #最大行数の変更
    return(display(df_return_latest))

In [142]:
for i in range(len(team_name_list)):
    show_dataframe(i)

,results,home,score,match review
42,2,0,2-2,今季最多のサポーターが駆け付けた札幌ドに吉報は届かなかった。札幌は3位以内確保のために勝利が絶対条件の中、序盤から優勢な展開に。ジェイのスーパーゴールもあり、21分までに2点のリードを奪う。しかし、歯車は徐々に狂い出していった。ミスからピンチを招く場面が増加すると、前半39分に1点を返され、嫌なムードのまま後半開始直後に同点弾を許す。その後は都倉を投入して巻き返しを図るも、焦りからゴール前の精度は上がらず。引き分けで終了し、ACL出場圏への浮上はかなわず、消化不良でシーズンは閉幕した。


,results,home,score,match review
45,0,0,3-2,神戸は今季の最終戦でホームのサポーターに勝利を届けた。やはりチームをリードしたのはアンドレスイニエスタとルーカスポドルスキ。2人を経由したパスワークでゲームをコントロールし、狭いスペースでも仙台の「嫌なところ」を突いて決定機を生み出した。さらに、最前線では古橋が起点となって2列目の選手を生かす効果的なプレーをするだけでなく、自らの仕掛けやゴール前に飛び込むプレーを披露するなど、終始相手を翻弄。守備面ではやや課題を残したものの、勝点を積み重ねて10位でのフィニッシュとなった。


,results,home,score,match review
55,2,0,0-0,ホームに鳥栖を迎えた鹿島。昌子の高精度な縦パスを軸に攻撃を組み立てるも、前線の鈴木が高橋秀と高橋義の徹底マークに遭い、決定機を作れない。頼みのサイド攻撃も遠藤と安西にボールが入らず、ファイナルサードを崩せないこう着した状況が続く。それでも、守備面では全員の集中した守備で、前節に決勝点を挙げた好調のフェルナンドトーレスに仕事をさせない。試合はそのままスコアレスでタイムアップを迎えるも、他会場の結果でACL出場権を獲得。今節の任務を完遂した。一方の鳥栖も残留が決まり、双方に悪くない結果となった。


,results,home,score,match review
45,1,0,3-2,大幅にスタメンを変更して最終戦に臨んだ浦和。代わって先発した選手らが役割を果たし、選手層の厚さを見せつける結果となった。その中で特に顕著な働きを見せたのが李とアンドリューナバウト。前半にセットプレーから李が押し込んで先制点を決めると、1点リードで迎えた後半23分には、アンドリューナバウトの突破から再び李が押し込んで決定的な3点目を奪取。その後は猛攻を受けたものの、懸命のディフェンスで逃げ切った。早期に優勝争いから脱落した悔しいシーズンとなったが、来季に向けて希望が持てる一戦となった。


,results,home,score,match review
46,1,0,4-2,「柏レイソルはどんな時でも強くなければならない」。スタンドに掲げられた言葉を体現したかのようなイレブンの戦いぶりに、三協F柏が沸いた。今季初の連勝を目指した柏は前節同様に選手の足並みがそろっており、攻守ともに迷いなく、ゴールへと直結するプレーを徹底。高速カウンター、豪快なミドルシュート、そして鮮やかな崩しと、躍進した昨季を思い起こさせるような多彩な攻撃を披露した。「太陽王」が見せた積極的な姿勢は、まさしく来季への指針だ。1年でJ1へと舞い戻るために、このスタイルを忘れずに来季の戦いへと臨みたい。


...

,results,home,score,match review
45,1,0,1-2,前節の黒星で16位との勝点差が1に縮まった横浜FM。今節は9試合連続勝利なしと相性の悪いC大阪をホームに迎えた。カウンターを狙う相手に対して思うように敵陣でボールを持てず、序盤から激しく攻守が入れ替わる展開に。10試合ぶりに採用した3バックの陣形や飯倉の好セーブが光り、スコアレスで折り返す。後半に入ると、天野や扇原が攻撃のタクトを振り、後半5分には先制に成功。しかし、同11分に同点に追い付かれ、その5分後にはCKから失点を許し、1－2の敗戦。約2ヵ月ぶりのホームでの白星を挙げられなかった。


,results,home,score,match review
44,1,0,3-2,神戸は今季の最終戦でホームのサポーターに勝利を届けた。やはりチームをリードしたのはアンドレスイニエスタとルーカスポドルスキ。2人を経由したパスワークでゲームをコントロールし、狭いスペースでも仙台の「嫌なところ」を突いて決定機を生み出した。さらに、最前線では古橋が起点となって2列目の選手を生かす効果的なプレーをするだけでなく、自らの仕掛けやゴール前に飛び込むプレーを披露するなど、終始相手を翻弄。守備面ではやや課題を残したものの、勝点を積み重ねて10位でのフィニッシュとなった。


,results,home,score,match review
42,2,0,2-2,今季最多のサポーターが駆け付けた札幌ドに吉報は届かなかった。札幌は3位以内確保のために勝利が絶対条件の中、序盤から優勢な展開に。ジェイのスーパーゴールもあり、21分までに2点のリードを奪う。しかし、歯車は徐々に狂い出していった。ミスからピンチを招く場面が増加すると、前半39分に1点を返され、嫌なムードのまま後半開始直後に同点弾を許す。その後は都倉を投入して巻き返しを図るも、焦りからゴール前の精度は上がらず。引き分けで終了し、ACL出場圏への浮上はかなわず、消化不良でシーズンは閉幕した。


,results,home,score,match review
43,2,0,0-0,ホームに鳥栖を迎えた鹿島。昌子の高精度な縦パスを軸に攻撃を組み立てるも、前線の鈴木が高橋秀と高橋義の徹底マークに遭い、決定機を作れない。頼みのサイド攻撃も遠藤と安西にボールが入らず、ファイナルサードを崩せないこう着した状況が続く。それでも、守備面では全員の集中した守備で、前節に決勝点を挙げた好調のフェルナンドトーレスに仕事をさせない。試合はそのままスコアレスでタイムアップを迎えるも、他会場の結果でACL出場権を獲得。今節の任務を完遂した。一方の鳥栖も残留が決まり、双方に悪くない結果となった。


,results,home,score,match review
41,2,1,4-4,「ゼイワンの航海第1章」は壮絶な内容で幕を閉じた。長崎はボールを奪ってから縦に速く、躍動感のある攻撃を仕掛けてトラスタの大歓声を引き出す。特に翁長は豊富な運動量で上下動を繰り返して攻守に躍動。ロングスローを武器にセットプレーでもゴールを演出した。しかし、守備面では課題を残す。連動性のある清水の仕掛けを止めることができず、一時は逆転を許してしまう。それでも、終盤のPKで追い付いてドローで決着。高木監督はチームを去るが、30の勝点とこの1年で得た経験は間違いなく来季につながるものとなった。
